# Installation and setup
##### (written on 4/10/2023 by Dr Shantanu Pathak)

**These Steps may change partially or completely with time**

### Step1 : Install boto3 using conda or pip on you PC / Laptop

conda install -c conda-forge boto3

ref: https://aws.amazon.com/sdk-for-python/


### Step2 : provide user credentials to boto3
So do following steps

a. login to aws account

b. Go to services -> all services -> IAM service --> Users

c. Click create user

d. give the name

e. give access permissions to group

4 access permissions need to be give
   Search in filter following and enable them
   EC2 full access
   IAM full access
   S3 full access
   Route53 full access

f. create group

g. assign the permissions

h. logout

### Step3. login go to iam service -> users
 a. select the user to give programming access

b. click security credentials

c. Select access keys

d. Select use case local code

e. click on confirmation message

f. click next

g. click download csv file

h. click done

i. check in security credentials , access keys status is active

j. sign out


### Step4. On your PC / laptop
a. create a directory AWS_S3

b. copy the downloaded credentials CSV file

c. create a .aws folder in 'C:\\Users\\your_user\\' folder

d. Inside .aws folder create file 'credentials' (without any extensions)

e. paste crendentials in follwing format

[default]

aws_access_key_id = YOUR_ACCESS_KEY_ID

aws_secret_access_key = YOUR_SECRET_ACCESS_KEY

f. Inside .aws folder create file 'config' (without any extensions)

g. paste region Mumbai in following format

[default]

region = ap-south-1

h. Now default profile for boto3 is setup!






# What can be done using Boto 3?
- Create and use any service of AWS
- Create S3 bucket
- Upload or Download files from S3 bucket
- Delete S3 Bucket

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/index.html

# Possible Actions to be performed on Bucket of S3

    copy
    create
    delete
    delete_objects
    download_file
    download_fileobj
    get_available_subresources
    load
    put_object
    upload_file
    upload_fileobj



# Create S3 Bucket Using Boto3

**Remember After creating a bucket you may be charged by AWS!**

## Naming Constraints for S3 bucket

AWS provides naming standards when naming an aws bucket.

- The bucket name can be between 3 and 63 characters long, and can **contain only lower-case characters, numbers, periods(dot), and dashes**.

- Each label in the bucket name must start with a lowercase letter or number.

- The bucket name **cannot contain underscores**, end with a dash, have consecutive periods, or use dashes adjacent to periods.

- The bucket name **cannot be** formatted as an **IP address** (198.51.100.24).

- **Remember** that this name must be **unique name throughout the whole AWS platform**, as bucket names are DNS compliant. If you try to create a bucket, but another user has already claimed your desired bucket name, your code will fail. Instead of success, you will see the error: botocore.errorfactory.BucketAlreadyExists.

Ref:
https://docs.aws.amazon.com/awscloudtrail/latest/userguide/cloudtrail-s3-bucket-naming-requirements.html


In [ ]:
import boto3

In [ ]:
import random
import string
import uuid
def create_valid_bucket_name(bucket_prefix,min_size=3,max_size=63):
    # The generated bucket name must be between 3 and 63 chars long
    # str(uuid.uuid4()) gives 36 characters randomly
    # from lowercase, hyphen and digits
    bname = ''.join([bucket_prefix, str(uuid.uuid4())])
    if max_size > len(bname) > min_size:
        size=len(bname)
    elif len(bname) < min_size :
        bname += random.choices(string.ascii_lowercase + string.digits,
                                max_size - len(bname))
    return bname[:max_size]

In [ ]:
# Verify the implementation of bucket name
# name = create_valid_bucket_name("as"*35)
name = create_valid_bucket_name("shantanu-pathak")
print(name, len(name))

shantanu-pathake98cd5ec-54ce-4b14-8882-c4e3c58427ff 51


In [ ]:
def create_bucket(bucket_prefix, s3_resource):
    session = boto3.session.Session()
    print("Current region is ", session.region_name, type(session.region_name))
    current_region = session.region_name
    bucket_name = create_valid_bucket_name(bucket_prefix)
    print("Bucket name is ", bucket_name)
    bucket_response = s3_resource.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
        'LocationConstraint': current_region})
    print(bucket_name, current_region)
    return bucket_name, bucket_response

In [ ]:
s3_resource = boto3.resource('s3', region_name='ap-south-1')
create_bucket('shantanu-pathak', s3_resource)
print("Hello, Amazon S3! Let's list your buckets:")
for bucket in s3_resource.buckets.all():
    print("Name of the bucket is : ", bucket.name)
print("Completed!")

Current region is  ap-south-1 <class 'str'>
Bucket name is  shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d
shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d ap-south-1
Hello, Amazon S3! Let's list your buckets:
Name of the bucket is :  shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d
Completed!


### Possible Errors in Creation of Bucket

##### BucketAlreadyOwnedByYou:

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

- Solution :: Change the name of bucket, you already have bucket with same name!

##### InvalidBucketName:
botocore.exceptions.ClientError: An error occurred (InvalidBucketName) when calling the CreateBucket operation: The specified bucket is not valid.

- Solution ::
- Check the name of bucket is valid / not as per valid name rules of AWS
- Check if region mensioned is valid or not for your login

    Mumbai region code is 'ap-south-1'
    
    List of AWS regions and their codes
    https://docs.aws.amazon.com/general/latest/gr/rande.html#s3_region

- Check if "'LocationConstraint': current_region" is given while creating a bucket


# List all your buckets

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')

print("Hello, Amazon S3! Let's list your buckets:")
for bucket in s3_resource.buckets.all():
    print("Name of the bucket is : ", bucket.name)
print("Completed!")

Hello, Amazon S3! Let's list your buckets:
Completed!


# Upload file to a bucket

- create / choose a file to be uploaded

- Here we have created a file "AWS_S3_info.txt"

- Get the local path of the file

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name= "shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d"
local_file_name = "I:\\ssp\\Python\\AWS_S3\\AWS_S3_info.txt"
s3_file_name="AWS_S3_info.txt"
s3_resource.Bucket(bucket_name).upload_file(
    Filename=local_file_name, Key=s3_file_name)
print("File upload completed!")

File upload completed!


# Download a file

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name= "shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d"
local_file_name = "I:\\ssp\\Python\\AWS_S3\\downloaded_AWS_S3_info.txt"
s3_file_name="AWS_S3_info.txt"

s3_resource.Object(bucket_name, s3_file_name).download_file(
    local_file_name)
print("File downloaded successfully!")

File downloaded successfully!


# List all File / Object names in a S3 Bucket

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name= "shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d"
bucket=s3_resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(obj.key)

AWS_S3_info.txt


### List all File / Object Details in a S3 Bucket

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name= "shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d"
bucket=s3_resource.Bucket(bucket_name)
print("Key","\tStorage_Class", "Last Modified", "\t\tVer_id", "Metadata", sep="\t")
for obj in bucket.objects.all():
    subsrc = obj.Object()
    print(obj.key, obj.storage_class, obj.last_modified,
          subsrc.version_id, subsrc.metadata, sep="\t")

# Delete a bucket

## Delete non-empty bucket

In [1]:
def delete_all_objects(s3_resource,bucket_name):
    res = []
    bucket=s3_resource.Bucket(bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})
    print("All objects in ", bucket_name , ' deleted!')


In [ ]:
import boto3
s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name="shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d"
# delete all objects in a bucket
delete_all_objects(s3_resource,bucket_name)
# delete the bucket
s3_resource.Bucket(bucket_name).delete()

[{'Key': 'AWS_S3_info.txt', 'VersionId': 'null'}]
All objects in  shantanu-pathak85eb410b-b409-43ec-8ab1-816b31eada8d  deleted!


{'ResponseMetadata': {'RequestId': 'M4P6HM06JAJMQW0P',
  'HostId': 'BX5yUptGqOOvl1oZszSQB+98ZfM7XVExST/a/Zj9BFAwgijp8bSFscqgk5JKu1Pclmm0msurTxI=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'BX5yUptGqOOvl1oZszSQB+98ZfM7XVExST/a/Zj9BFAwgijp8bSFscqgk5JKu1Pclmm0msurTxI=',
   'x-amz-request-id': 'M4P6HM06JAJMQW0P',
   'date': 'Wed, 04 Oct 2023 10:54:11 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Delete ALL (Empty or  Non-empty) buckets

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
print("Let's delete ALL (Empty or Non-Empty) buckets:")
for bucket in s3_resource.buckets.all():
    print("delete all objects in a bucket")
    delete_all_objects(s3_resource,bucket.name)
    print("Deleting the bucket : ", bucket.name)
    s3_resource.Bucket(bucket.name).delete()

Let's delete ALL your buckets:
delete all objects in a bucket
[{'Key': 'AWS_S3_info.txt', 'VersionId': 'null'}]
All objects in  shantanu-bucket1ddcf2aae-b615-4db1-bb95-2064aa39598c  deleted!
Deleting the bucket :  shantanu-bucket1ddcf2aae-b615-4db1-bb95-2064aa39598c
delete all objects in a bucket
[{'Key': 'AWS_S3_info.txt', 'VersionId': 'null'}]
All objects in  shantanu-bucket1ed042d09-454b-43a0-8c6e-8538a4349ba8  deleted!
Deleting the bucket :  shantanu-bucket1ed042d09-454b-43a0-8c6e-8538a4349ba8
delete all objects in a bucket
[{'Key': 'AWS_S3_info.txt', 'VersionId': 'null'}]
All objects in  shantanu-bucket29b04bd87-1079-4d4d-a5fa-1735fbb915de  deleted!
Deleting the bucket :  shantanu-bucket29b04bd87-1079-4d4d-a5fa-1735fbb915de
delete all objects in a bucket
[{'Key': 'AWS_S3_info.txt', 'VersionId': 'null'}]
All objects in  shantanu-bucket2f5fa909b-35b7-4306-b0ac-e4bbfe338607  deleted!
Deleting the bucket :  shantanu-bucket2f5fa909b-35b7-4306-b0ac-e4bbfe338607


# Delete Empty Bucket

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket_name = "shantanu-pathak"
s3_resource.Bucket(bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'RDMX4QWNYRT2BQJ7',
  'HostId': 'J5VcHHljUTJoUIm6OWsW7fP8CS/5Ox5mFhsiEoaAIIHvCREfBnIXu5tMqhQ7BU3BrzKV58uvLdU=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'J5VcHHljUTJoUIm6OWsW7fP8CS/5Ox5mFhsiEoaAIIHvCREfBnIXu5tMqhQ7BU3BrzKV58uvLdU=',
   'x-amz-request-id': 'RDMX4QWNYRT2BQJ7',
   'date': 'Wed, 04 Oct 2023 10:28:58 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Delete All Empty S3 Buckets

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
print("Let's delete ALL your EMPTY buckets:")
for bucket in s3_resource.buckets.all():
    print("Deleting the bucket : ", bucket.name)
    bucket.delete()
print("All buckets deleted!")

Let's delete ALL your buckets:
All buckets deleted!


## Possible Errors in Deleting a bucket

##### NoSuchBucket
    NoSuchBucket: An error occurred (NoSuchBucket) when calling the
    ListObjectVersions operation: The specified bucket does not exist
        
    Solution: Name of the bucket is wrong or doesn't exist or already deleted!
        

# Copy file from one bucket to other bucket

Steps :

- Create two S3 buckets
- Upload a file to bucket 1
- Copy file from bucket1 to 2
- Check in bucket 2 if file uploaded successfully!

In [2]:
def copy_to_bucket(s3_resource,bucket_from_name, bucket_to_name, file_name):
    copy_source = {
        'Bucket': bucket_from_name,
        'Key': file_name
    }
    s3_resource.Object(bucket_to_name, file_name).copy(copy_source)

In [ ]:
import boto3

s3_resource = boto3.resource('s3', region_name='ap-south-1')
bucket1_name, bucket_response = create_bucket('shantanu-bucket1', s3_resource)
bucket2_name, bucket_response = create_bucket('shantanu-bucket2', s3_resource)
print("####################")
print("Check if buckets created. Following is list all buckets created :")
for bucket in s3_resource.buckets.all():
    print("Name of the bucket is : ", bucket.name)

print("####################")
print("Uploading file to bucket 1 --> ")
local_file_name = "I:\\ssp\\Python\\AWS_S3\\AWS_S3_info.txt"
s3_file_name="AWS_S3_info.txt"

s3_resource.Bucket(bucket1_name).upload_file(
    Filename=local_file_name, Key=s3_file_name)
print("File uploaded successfully successfully!")

print("####################")
print("Copy file from bucket1 to bucket2")

copy_to_bucket(s3_resource,bucket1_name, bucket2_name, s3_file_name)

print("####################")
print("Check all file names in Bucket 2")

bucket=s3_resource.Bucket(bucket2_name)
print("Key","\tStorage_Class", "Last Modified", "\t\tVer_id", "Metadata", sep="\t")
for obj in bucket.objects.all():
    print(obj.key)
print("Completed!")

Current region is  ap-south-1 <class 'str'>
Bucket name is  shantanu-bucket1ddcf2aae-b615-4db1-bb95-2064aa39598c
shantanu-bucket1ddcf2aae-b615-4db1-bb95-2064aa39598c ap-south-1
Current region is  ap-south-1 <class 'str'>
Bucket name is  shantanu-bucket2f5fa909b-35b7-4306-b0ac-e4bbfe338607
shantanu-bucket2f5fa909b-35b7-4306-b0ac-e4bbfe338607 ap-south-1
####################
Check if buckets created. Following is list all buckets created :
Name of the bucket is :  shantanu-bucket1ddcf2aae-b615-4db1-bb95-2064aa39598c
Name of the bucket is :  shantanu-bucket1ed042d09-454b-43a0-8c6e-8538a4349ba8
Name of the bucket is :  shantanu-bucket29b04bd87-1079-4d4d-a5fa-1735fbb915de
Name of the bucket is :  shantanu-bucket2f5fa909b-35b7-4306-b0ac-e4bbfe338607
####################
Uploading file to bucket 1 --> 
File uploaded successfully successfully!
####################
Copy file from bucket1 to bucket2
####################
Check all file names in Bucket 2
Key		Storage_Class	Last Modified			Ver_id	Me

#### Further reading

- https://realpython.com/python-boto3-aws-s3/#deleting-a-non-empty-bucket

- https://docs.aws.amazon.com/general/latest/gr/rande.html#s3_region

- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/bucket/index.html

- https://saturncloud.io/blog/how-to-upload-to-amazon-s3-using-boto3-and-return-public-url/